In [1]:
from langgraph.graph import START, StateGraph, END
from typing import Literal, List
from typing_extensions import TypedDict
import os
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_groq import ChatGroq
import cv2
from deepface import DeepFace
import csv
from datetime import datetime
import json

In [8]:
llm.invoke("Who are you?").content

'I\'m an artificial intelligence model known as Llama. Llama stands for "Large Language Model Meta AI."'

In [9]:
load_dotenv()

GROQ_API_KEY = os.getenv("GROQ_API_KEY")
llm = ChatGroq(model_name="llama-3.3-70b-versatile", temperature=0.1, api_key=GROQ_API_KEY)

In [10]:
class GraphState(TypedDict):
    questions: str
    desc: str
    emotions: List[dict]
    annotations: List[dict]

In [11]:
from typing import List, TypedDict
import cv2
from deepface import DeepFace
import csv
from datetime import datetime

def emotion_detection(state: GraphState):
    cap = cv2.VideoCapture(0)

    while cap.isOpened():
        ret, frame = cap.read()
        if ret:
            print("FRAME CAPTURED SUCCESSFULLY")
            try:
                result = DeepFace.analyze(frame, actions=["emotion"], enforce_detection=False)

                print("Analysis Result:", result)

                if isinstance(result, list):
                    result = result[0]

                if "emotion" in result:
                    emotion = result["dominant_emotion"]
                    confidence = result["emotion"][emotion]

                    if confidence > 70:
                        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

                        state["emotions"].append({
                            "timestamp": timestamp,
                            "emotion": emotion,
                            "confidence": confidence
                        })

                        with open("emotions.csv", mode="a", newline="") as file:
                            writer = csv.writer(file)
                            writer.writerow([timestamp, emotion, confidence])

                        cv2.putText(frame, f"Emotion: {emotion} ({confidence:.2f}%)", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

                        if "region" in result:
                            x, y, w, h = result["region"]["x"], result["region"]["y"], result["region"]["w"], result["region"]["h"]
                            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    else:
                        print(f"Confidence is below 70%. Emotion: {emotion}, Confidence: {confidence:.2f}%")

                else:
                    print("No emotion data found in the result.")

            except Exception as e:
                print(f"An exception has occurred: {e}")

            cv2.imshow("Emotion Recognition", frame)

        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    cap.release()
    cv2.destroyAllWindows()


In [12]:
def annotations_node(state):
    annotations = state["annotations"]

    for entry in state["emotions"]:
        timestamp, emotion, confidence = entry
        annotation = {
            "timestamp": timestamp,
            "emotion": emotion,
            "confidence": confidence,
        }
        annotations.append(annotation)
    
    with open("annotations.json", "w") as file:
        json.dump(annotations, file, indent=4)
    
    print("Annotations created and saved successfully.")
    
    return {"annotations" : annotations}


In [14]:
def generate_question(state : GraphState):
    questions = state["questions"]
    desc = state["desc"]

    prompt = f"""
        You are an expert interviewer tasked with creating tailored interview questions based on the provided job description. Your goal is to assess the candidate's knowledge, skills, and problem-solving abilities relevant to the role of a Machine Learning Engineer.

        **Instructions:**
        - Generate questions that are direct, specific, and aligned with the job description.
        - Divide the questions into three levels of difficulty:
        1. Easy: Basic concepts and introductory questions.
        2. Medium: Practical applications and intermediate-level problems.
        3. Hard: Advanced, analytical, and scenario-based challenges.
        - Include a mix of theoretical, coding, and situational questions.
        - Ensure clarity and precision in each question.

        **Job Description:**
        {desc}
        NOTE : ONLY FORM 2 QUESTIONS IN EACH CATEGORY.
        **Output Format:**
        1. Easy Level:
        - Question 1:
        - Question 2:
        ...
        2. Medium Level:
        - Question 1:
        - Question 2:
        ...
        3. Hard Level:
        - Question 1:
        - Question 2:
        ...
    """

    
    prompt_template = PromptTemplate(input_variables=["job_description"], template=prompt)

    chain = LLMChain(llm=llm, prompt=prompt_template)

    questions = chain.run(job_description=desc)

    return{"questions" : questions}



In [19]:
workflow = StateGraph(GraphState)

workflow.add_node("emotions_detections", emotion_detection)
workflow.add_node("annotations_node", annotations_node)
workflow.add_node("generate_question", generate_question)

workflow.add_edge(START, "emotion_detections")
workflow.add_edge("emotion_detections", "annotations_node")
workflow.add_edge("annotations_node", "generate_question")
workflow.add_edge("generate_question", END)

app = workflow.compile()

ValueError: Found edge starting at unknown node 'emotion_detections'